In [1]:
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
from scipy.spatial import distance
import os
# from sklearn.cluster import KMeans
# from sklearn.metrics import adjusted_rand_score, silhouette_score
from scipy import stats as st
from mpl_toolkits import mplot3d
import plotly.graph_objs as go
import copy

In [2]:
def dist(A,B,k):
  # print(A)
  # print(B)
  dis=0
  
  for i in range(0,len(A)):
    dis+=(A[i]-B[i])*(A[i]-B[i])
  # print(dis)    
  return dis  

In [3]:

def Distance(cluster1,cluster2,k):
  Dis=0
  if k==1:
    Dis=10000000000000   #min
    for i in cluster1:
      for j in cluster2:
        Dis=min(Dis,dist(i,j,k))
  elif k==2:      
    Dis=0             #max
    for i in cluster1:
      for j in cluster2:
        Dis=max(Dis,dist(i,j,k))
  elif k==3:      
    Dis=0             #avg
    cnt=0
    for i in cluster1:
      for j in cluster2:
        # print(i," ",j,"------------")
        Dis+=dist(i,j,k)
        cnt+=1
    Dis/=cnt    
  return Dis    

In [7]:
def linkage(X,k,l):
  cluster=[]
  for i in range (0,len(X)):
    cluster.append(i)
    
  for abc in range(0,len(X)-k):
    # print(cluster)
    N=len(set(cluster))
    Dis=1000000000
    ind1=-1
    ind2=-1
    for i in range(0,N):
      for j in range(i+1,N):
        lis1=[]
        lis2=[]
        for m in range(0,len(cluster)):
          if cluster[m]==i:
            lis1.append(X[m])
          elif cluster[m]==j:
            lis2.append(X[m])  
        # print(lis1," ",lis2," ",Distance(lis1,lis2,l))
        if Distance(lis1,lis2,l)<Dis:
          Dis=Distance(lis1,lis2,l)
          ind1=i
          ind2=j
    # print(ind1," ",ind2," ",Dis)      
    for i in range(0,len(cluster)):
      if cluster[i]==ind2:
        cluster[i]=ind1
      elif cluster[i]>ind2:
        cluster[i]-=1  

        
  return cluster  



In [18]:
import plotly.graph_objs as go
from plotly.subplots import make_subplots
import numpy as np

def plot(datasets, labels):
    #colour pattern
    colorscale = ['white','red','yellow','green','orange','violet','blue']

    # Define the layout for the plot
    layout = go.Layout(
        margin=dict(l=0, r=0, b=0, t=0),
        scene=dict(
            xaxis=dict(title='X'),
            yaxis=dict(title='Y'),
            zaxis=dict(title='Z'),
            aspectmode='data',
            aspectratio=dict(x=1, y=1, z=1),
            bgcolor='black',
           
            camera=dict(
                up=dict(x=0, y=0, z=1),
            ),
        ),
        paper_bgcolor='black',
    )

    #figure 1
    trace = go.Scatter3d(
        x=datasets[:, 0],
        y=datasets[:, 1],
        z=datasets[:, 2],
        mode='markers',
        marker=dict(
            color="white",
            size=3,
            opacity=1,
            
        ),
    )

    #figure 2
    trace1 = go.Scatter3d(
        x=datasets[:,0],
        y=datasets[:,1],
        z=datasets[:,2],
        mode='markers',
        marker=dict(
            color=labels,
            colorscale=colorscale,
            size=3,
            opacity=1,
            
        ),
    )

    #figure 3
    # dataset=datasets[np.where(labels!=-1)]
    # label=labels[np.where(labels!=-1)]
    # colorscale=colorscale[1:]
    # trace2 = go.Scatter3d(
    #     x=dataset[:,0],
    #     y=dataset[:,1],
    #     z=dataset[:,2],
    #     mode='markers',
    #     marker=dict(
    #         color=label,
    #         colorscale=colorscale,
    #         size=1,
    #         opacity=1,
            
    #     ),
    # )
    fig = make_subplots(rows=1, cols=2, specs=[[{'type': 'scatter3d'}]*2]*1)
    fig.update_layout(layout)

    fig.add_trace(trace, row=1, col=1)
    fig.add_trace(trace1, row=1, col=2)
    # fig.add_trace(trace2, row=1, col=3)
    
    for template in ["plotly", "plotly_white", "plotly_dark", "ggplot2", "seaborn", "simple_white", "none"]:
        fig.update_layout(template=template)
    fig.show()    
    return fig
 

In [22]:
path="../ds2.csv"
df = pd.read_csv(path, sep="\t", header=None)
X, labels = df.iloc[:, :-1], df.iloc[:, -1]
if(X.shape[1]==0):
  df = pd.read_csv(path, sep=",", header=None)
  X, labels = df.iloc[:, :-1], df.iloc[:, -1]
if(X.shape[1]==0):
  df = pd.read_csv(path, sep=" ", header=None)
  X, labels = df.iloc[:, :-1], df.iloc[:, -1]
 # if X.shape[1]==0:
 #   print(X)
X, labels = np.array(X), np.array(labels)
# print(X)
K=len(set(labels))
print(X) 
min_cluster=linkage(copy.copy(X),K,3)
max_cluster=linkage(copy.copy(X),K,3)
avg_cluster=linkage(copy.copy(X),K,3)
originial_fig=plot(X,labels)
max_linkage_fig=plot(X,max_cluster)
min_linkage_fig=plot(X,min_cluster)
avg_linkage_fig=plot(X,avg_cluster)
originial_fig.show()
originial_fig.write_html("cluster/"+path[3:6]+"original.html")
max_linkage_fig.show()
max_linkage_fig.write_html("cluster/"+path[3:6]+"maxLinkage.html")
min_linkage_fig.show()
min_linkage_fig.write_html("cluster/"+path[3:6]+"minLinkage.html")
avg_linkage_fig.show()
avg_linkage_fig.write_html("cluster/"+path[3:6]+"avgLinkage.html")

[[0.1 0.2 0.3]
 [0.2 0.1 0.4]
 [0.3 0.3 0.1]
 [0.4 0.5 0.6]
 [0.5 0.7 0.8]
 [0.6 0.6 0.9]
 [0.7 0.4 0.7]
 [0.8 0.3 0.6]
 [0.9 0.4 0.4]
 [0.2 0.8 0.2]
 [0.4 0.9 0.4]
 [0.6 0.8 0.1]
 [0.8 0.7 0.2]
 [0.1 0.5 0.7]
 [0.3 0.4 0.9]
 [0.5 0.3 0.8]
 [0.7 0.2 0.5]
 [0.9 0.1 0.2]
 [0.2 0.1 0.5]
 [0.4 0.3 0.3]
 [0.6 0.5 0.2]
 [0.8 0.6 0.1]
 [0.1 0.8 0.5]
 [0.3 0.7 0.4]
 [0.5 0.6 0.3]]


In [23]:
print(K)

3
